In [5]:
import pandas as pd

data = pd.read_csv('e:/Github-Repositories/datasets/truedata/electricity/electricity.txt',header=None,sep=',').values[800:900]

data.shape



(100, 321)

In [ ]:
import os
import numpy as np
import torch


from lightning.pytorch.trainer import Trainer
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from dsanet_model import DSANet
from dsanet_model import add_model_config
from data import MTSFDataset
from torch.utils.data import DataLoader
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import warnings

warnings.filterwarnings("ignore")

SEED = 7
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

root_dir = "/root/myDL/DSANet"
log_dir = os.path.join(root_dir, "dsanet_logs")

config = add_model_config(root_dir)

model = DSANet(config)
# print(model)
tb_logger = TensorBoardLogger(name="electricity", save_dir=config["log_dir"], version=2)
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.001, patience=3, verbose=False, mode="max")


trainer = Trainer(
    devices=1,
    logger=tb_logger,
    max_epochs=20,
    callbacks=[early_stop_callback]
)

trainer.fit(model)


In [ ]:
trainer.test()

In [ ]:
trainer.early_stopping_callback

In [ ]:
# 加载继续训练
import lightning.pytorch as pl

csv_logger = pl.loggers.CSVLogger(save_dir="lightning_logs/csi", name="version_0")
trainer = pl.Trainer(
    logger = csv_logger,
    callbacks=[pl.callbacks.ModelCheckpoint(every_n_train_steps=60, save_top_k=-1)],
)

model = DSANet(config)
ckp1 = "lightning_logs/csi/version_3/checkpoints/epoch=19-step=4280.ckpt"
ckp2 = "lightning_logs/csi/version_4/checkpoints/epoch=39-step=8560.ckpt"
ckp3 = 'lightning_logs/csi/version_0/checkpoints/epoch=4-step=1070.ckpt'
trainer.test(
    model=model,
    ckpt_path=ckp3,
)

In [ ]:
PATH = "lightning_logs/csi/version_4/checkpoints/epoch=39-step=8560.ckpt"
model = DSANet.load_from_checkpoint(PATH, config=config)

print(model.config["learning_rate"])
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
x = torch.randn(1,64, 32).to(device)
with torch.no_grad():
    y_hat = model(x)
print(y_hat)

In [ ]:
x.shape

In [ ]:
trainer = pl.Trainer(
    callbacks=[pl.callbacks.ModelCheckpoint(every_n_train_steps=60, save_top_k=-1)],
)

model = DSANet(config)


In [ ]:
import tensorboardX